In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practice').getOrCreate()

This is the [Demographics Data Analyzer project]('https://www.freecodecamp.org/learn/data-analysis-with-python/data-analysis-with-python-projects/demographic-data-analyzer') by freeCodeCamp, but done using pyspark

In [2]:
df = spark.read.csv('adult.data.csv', header=True, inferSchema=True)
df

DataFrame[age: int, workclass: string, fnlwgt: int, education: string, education-num: int, marital-status: string, occupation: string, relationship: string, race: string, sex: string, capital-gain: int, capital-loss: int, hours-per-week: int, native-country: string, salary: string]

In [3]:
df.show(5)

+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
|age|       workclass|fnlwgt|education|education-num|    marital-status|       occupation| relationship| race|   sex|capital-gain|capital-loss|hours-per-week|native-country|salary|
+---+----------------+------+---------+-------------+------------------+-----------------+-------------+-----+------+------------+------------+--------------+--------------+------+
| 39|       State-gov| 77516|Bachelors|           13|     Never-married|     Adm-clerical|Not-in-family|White|  Male|        2174|           0|            40| United-States| <=50K|
| 50|Self-emp-not-inc| 83311|Bachelors|           13|Married-civ-spouse|  Exec-managerial|      Husband|White|  Male|           0|           0|            13| United-States| <=50K|
| 38|         Private|215646|  HS-grad|            9|          Divorced|Handlers-cleaners|Not-i

In [4]:
df.columns

['age',
 'workclass',
 'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country',
 'salary']

## How many people of each race are represented in this dataset?

In [5]:
df.groupby('race').count().show()

+------------------+-----+
|              race|count|
+------------------+-----+
|             Other|  271|
|Amer-Indian-Eskimo|  311|
|             White|27816|
|Asian-Pac-Islander| 1039|
|             Black| 3124|
+------------------+-----+



## What is the average age of men?

In [6]:
df.filter(df.sex=='Male').groupBy().avg('age').show()

+-----------------+
|         avg(age)|
+-----------------+
|39.43354749885268|
+-----------------+



## What is the percentage of people who have a Bachelor's degree?

In [7]:
(df.filter(df.education == 'Bachelors').count()/df.count()) * 100

16.44605509658794

## What percentage of people with advanced education (Bachelors, Masters, or Doctorate) make more than 50K?

In [8]:
df.filter(((df.education == 'Bachelors') |
                     (df.education == 'Masters') |
                     (df.education == 'Doctorate')) & (df.salary == ">50K")).count()/df.count() * 100

10.706059396210192

## What percentage of people without advanced education make more than 50K?

In [9]:
df.filter(~((df.education == 'Bachelors') |
                     (df.education == 'Masters') |
                     (df.education == 'Doctorate')) & (df.salary == ">50K")).count()/df.count() * 100

13.374896348392248

## What is the minimum number of hours a person works per week?

In [10]:
minimum = df.groupBy().min('hours-per-week').collect()[0][0]
minimum

1

## What percentage of the people who work the minimum number of hours per week have a salary of more than 50K?

In [11]:
df.filter((df['hours-per-week'] == minimum) & (df['salary'] == ">50K")).count()/df.filter(df['hours-per-week'] == minimum).count() * 100

10.0

## What country has the highest percentage of people that earn >50K and what is that percentage?

In [12]:
rich = df.filter(df['salary'] == ">50K").groupby('native-country').count().withColumnRenamed('count', 'rich')

In [13]:
countries = df.groupby('native-country').count()

In [14]:
from pyspark.sql.functions import *

df2 = rich.join(countries, 'native-country').withColumn("percentage", 
                                                        (col("rich") / col("count")))

In [15]:
df2.sort(col('percentage').desc()).select(['native-country', 'percentage']).show(1)

+--------------+------------------+
|native-country|        percentage|
+--------------+------------------+
|          Iran|0.4186046511627907|
+--------------+------------------+
only showing top 1 row



## Identify the most popular occupation for those who earn >50K in India.

In [16]:
df.filter((df['salary'] == ">50K") & (df['native-country'] == 'India')).groupby('occupation').count().sort(col('count').desc()).show(1)

+--------------+-----+
|    occupation|count|
+--------------+-----+
|Prof-specialty|   25|
+--------------+-----+
only showing top 1 row

